In [4]:
import pandas as pd

load the dataset

In [5]:
df = pd.read_csv('Sample Dataset.csv')


CHECK FOR BASIC INFOS 


In [7]:
print(df.head()) # display the first few rows of the dataframe
print(df.shape) # display the shape of the dataframe
print(df.info()) # display the summary information of the dataframe
print(df['Price'].describe()) # display the statistical summary of the 'Price' column
print(df.isnull().sum()) # display the count of missing values in each column 

           Brand  Year   Model               Car/Suv  \
0      Ssangyong  2022  Rexton  Sutherland Isuzu Ute   
1             MG  2022     MG3             Hatchback   
2            BMW  2022    430I                 Coupe   
3  Mercedes-Benz  2011    E500                 Coupe   
4        Renault  2022  Arkana                   SUV   

                                       Title UsedOrNew Transmission  \
0       2022 Ssangyong Rexton Ultimate (awd)      DEMO    Automatic   
1  2022 MG MG3 Auto Excite (with Navigation)      USED    Automatic   
2                      2022 BMW 430I M Sport      USED    Automatic   
3           2011 Mercedes-Benz E500 Elegance      USED    Automatic   
4                 2022 Renault Arkana Intens      USED    Automatic   

         Engine DriveType  FuelType FuelConsumption  Kilometres  \
0  4 cyl, 2.2 L       AWD    Diesel  8.7 L / 100 km        5595   
1  4 cyl, 1.5 L     Front   Premium  6.7 L / 100 km          16   
2    4 cyl, 2 L      Rear   Premium

SELECT ONLY USEFUL ONES

In [8]:
features = ['Brand', 'Year', 'Model', 'Car/Suv', 'UsedOrNew', 
            'Transmission', 'Engine', 'DriveType', 'FuelType', 
            'FuelConsumption', 'Kilometres', 'Location', 'CylindersinEngine']

target = 'Price'

CREATE SMALLER DATAFRAMES

In [10]:
df_model = df[features + [target]].copy()
print(df_model.head())
print(df_model.isnull().sum())

           Brand  Year   Model               Car/Suv UsedOrNew Transmission  \
0      Ssangyong  2022  Rexton  Sutherland Isuzu Ute      DEMO    Automatic   
1             MG  2022     MG3             Hatchback      USED    Automatic   
2            BMW  2022    430I                 Coupe      USED    Automatic   
3  Mercedes-Benz  2011    E500                 Coupe      USED    Automatic   
4        Renault  2022  Arkana                   SUV      USED    Automatic   

         Engine DriveType  FuelType FuelConsumption  Kilometres  \
0  4 cyl, 2.2 L       AWD    Diesel  8.7 L / 100 km        5595   
1  4 cyl, 1.5 L     Front   Premium  6.7 L / 100 km          16   
2    4 cyl, 2 L      Rear   Premium  6.6 L / 100 km        8472   
3  8 cyl, 5.5 L      Rear   Premium   11 L / 100 km      136517   
4  4 cyl, 1.3 L     Front  Unleaded    6 L / 100 km        1035   

            Location CylindersinEngine   Price  
0     Caringbah, NSW             4 cyl   51990  
1     Brookvale, NSW    

Handle the Text Data

#ONLY USING THE TEXT 
Already numerical (easy to use):

Year
Kilometres

Text but SIMPLE (few unique values):

Transmission (probably just "Automatic" and "Manual")
UsedOrNew (probably "USED", "NEW", "DEMO")
Car/Suv (probably just "Car" and "SUV")

Text and COMPLEX (many unique values):

Brand (30+ different brands)
Model (100+ different models)
Location (50+ different locations)
Engine (many variations like "4 cyl, 2.0 L")

In [11]:
from sklearn.preprocessing import LabelEncoder

In [13]:
numerical_features = ['Year','Kilometres']
categorical_features = ['Transmission', 'UsedOrNew', 'FuelType', 'DriveType']
target = 'Price'

In [14]:
df_model = df[numerical_features + categorical_features + [target]].copy()

In [15]:
print("Selected features:")
print(df_model.head())
print("\nData types:")
print(df_model.dtypes)

Selected features:
   Year  Kilometres Transmission UsedOrNew  FuelType DriveType   Price
0  2022        5595    Automatic      DEMO    Diesel       AWD   51990
1  2022          16    Automatic      USED   Premium     Front   19990
2  2022        8472    Automatic      USED   Premium      Rear  108988
3  2011      136517    Automatic      USED   Premium      Rear   32990
4  2022        1035    Automatic      USED  Unleaded     Front   34990

Data types:
Year             int64
Kilometres       int64
Transmission    object
UsedOrNew       object
FuelType        object
DriveType       object
Price            int64
dtype: object


In [16]:
print("\nUnique values in categorical features:")
for col in categorical_features:
    print(f"{col}: {df_model[col].unique()}")


Unique values in categorical features:
Transmission: ['Automatic' 'Manual']
UsedOrNew: ['DEMO' 'USED' 'NEW']
FuelType: ['Diesel' 'Premium' 'Unleaded' 'Hybrid' '-']
DriveType: ['AWD' 'Front' 'Rear' 'Other' '4WD']


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [49]:
print("Data after one-hot encoding:")
print(df_model.head())
print(df_model.columns.tolist())

print("Total columns after encoding:", len(df_model.columns))

Data after one-hot encoding:
   Year  Kilometres   Price  Transmission_Manual  UsedOrNew_NEW  \
0  2022        5595   51990                False          False   
1  2022          16   19990                False          False   
2  2022        8472  108988                False          False   
3  2011      136517   32990                False          False   
4  2022        1035   34990                False          False   

   UsedOrNew_USED  FuelType_Diesel  FuelType_Hybrid  FuelType_Premium  \
0           False             True            False             False   
1            True            False            False              True   
2            True            False            False              True   
3            True            False            False              True   
4            True            False            False             False   

   FuelType_Unleaded  DriveType_AWD  DriveType_Front  DriveType_Other  \
0              False           True            False    

# Separate features (X) and target (y)

In [50]:
X = df_model.drop('Price', axis=1)
Y = df_model['Price']

print("Features (X) shape:", X.shape)
print("Target (Y) shape:", Y.shape)
print("First few rows of features (X):", X.head())

Features (X) shape: (99, 13)
Target (Y) shape: (99,)
First few rows of features (X):    Year  Kilometres  Transmission_Manual  UsedOrNew_NEW  UsedOrNew_USED  \
0  2022        5595                False          False           False   
1  2022          16                False          False            True   
2  2022        8472                False          False            True   
3  2011      136517                False          False            True   
4  2022        1035                False          False            True   

   FuelType_Diesel  FuelType_Hybrid  FuelType_Premium  FuelType_Unleaded  \
0             True            False             False              False   
1            False            False              True              False   
2            False            False              True              False   
3            False            False              True              False   
4            False            False             False               True   

   Driv

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)
print("Training set:",len(X_train))
print("Testing set:",len(X_test))

Training set: 79
Testing set: 20


In [52]:
model = LinearRegression()
model.fit(X_train, y_train)
print("Model training completed.")

Model training completed.


In [53]:
y_pred = model.predict(X_test)

In [54]:
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))  


In [55]:

print("MODEL PERFORMANCE")
print(f"Mean Absolute Error: ${mae:,.2f}")
print(f"R² Score: {r2:.4f}")

print(f"Root Mean Squared Error: ${rmse:,.2f}")

MODEL PERFORMANCE
Mean Absolute Error: $7,761.48
R² Score: 0.5878
Root Mean Squared Error: $11,167.75


In [56]:
print("SAMPLE PREDICTIONS (First 15 Records)")
comparision = pd.DataFrame({
    'Actual Price': y_test.values[:15],
    'Predicted Price': y_pred[:15],
    'Difference': y_test.values[:15] - y_pred[:15]
})
print(comparision)

SAMPLE PREDICTIONS (First 15 Records)
    Actual Price  Predicted Price    Difference
0           9990      7503.017823   2486.982177
1          32900     23032.547920   9867.452080
2          26880     31286.548224  -4406.548224
3          13999     28857.672402 -14858.672402
4          16999     19010.585909  -2011.585909
5          24888     27856.421876  -2968.421876
6          24999     28207.539433  -3208.539433
7           9999      5070.157519   4928.842481
8          42500     14185.658693  28314.341307
9          51990     50825.464201   1164.535799
10         24880     18899.367088   5980.632912
11         22990     32576.396621  -9586.396621
12         19900     25211.395646  -5311.395646
13          8499      2119.230107   6379.769893
14         22750     21284.018533   1465.981467
